First, establish an ssh tunnel [with your PNI password] to jtb3-dev.princeton.edu to make the database available to you:

    ssh username@jtb3-dev.princeton.edu -L 3306:127.0.0.1:3306 -N

jtb3-dev is the database server on which all behavior and mouse data is stored. Second, make sure you have installed datajoint >= '0.12.dev4'
[ pip install datajoint==0.12.dev4 ]

In [ ]:
import pylab as pl
import datajoint as dj
import numpy as np
dj.__version__

# Work with tables directly

In [ ]:
dj.config['database.port'] = 6306

In [ ]:
#Overview across database schemas
dj.list_schemas()

In [ ]:
#Single entity relationship diagram. This is how the data is organized
acqu = dj.schema('pni_acquisition')
subj = dj.schema('pni_subject')
lab = dj.schema('pni_lab')

dj.ERD(subj)

# Reconstruct classes from the database directly

In [ ]:
acquisition = dj.create_virtual_module('acquisition', 'pni_acquisition')
subject = dj.create_virtual_module('subject', 'pni_subject')
action = dj.create_virtual_module('action', 'pni_action')
lab = dj.create_virtual_module('lab', 'pni_lab')
task = dj.create_virtual_module('task', 'pni_task')

In [ ]:
dj.Diagram(subject)

Now let's have some serious fun with the database, with some queries

In [ ]:
lab.User.describe();

In [ ]:
new_user = dict(
    user_id='testuser3',
    contact_via='Slack',
    presence='Away',
    day_cutoff_time=' '
)

In [ ]:
subject.Subject & 'user_id="ben"'

In [ ]:
acquisition.TowersBlock & 'user_id="ben"'

In [ ]:
dj.Table._update(lab.User & 'user_id="testuser3"', 'email', 'bla@princeton.edu')

In [ ]:
user = lab.User() & 'user_id="testuser3"'

In [ ]:
dj.kill()

In [ ]:
user.delete()

In [ ]:
subject.Subject.describe();

In [ ]:
lab.User()

In [ ]:
# Very fat mice
subject.Subject() & 'initial_weight > 35'

In [ ]:
subject.Subject() & 'dob between "2019-01-01" and "2019-03-01"'

In [ ]:
q = (subject.Subject & 'subject_id="B205"').proj('sex', 'dob', 'initial_weight', animal_location='location')

In [ ]:
result[0]

In [ ]:
# On which days of week does people like to work harder?
Session = acquisition.Session();
for day in range(7):
    print(len(Session & 'WEEKDAY(session_date) = ' + str(day)))

# Do fatter mice do better?

In [ ]:
# grab a mouse first
subj = subject.Subject & 'subject_id="E47"'
subj

In [ ]:
action.Weighing()

In [ ]:
# some magic to get weight on the date of each session
weight_with_date = action.Weighing.proj('weight', session_date="DATE(weighing_time)")

In [ ]:
weight_with_date * acquisition.Session

In [ ]:
# join weight_with_date and session to get both information
query = weight_with_date * acquisition.Session

In [ ]:
weight, performance = (query & 'subject_id = "E47"').fetch('weight','session_performance')

In [ ]:
weight

In [ ]:
pl.scatter(weight, performance)
pl.xlabel('bodyweight [g]')
pl.ylabel('performance')

## Are performance and weight gender dependent?

In [ ]:
subjects = subject.Subject.proj('sex') & query & 'user_id = "edward"'
corrs = []
for s in subjects:
    # fetch all related info: weight, performance and sex
    sex, weight, performance = (subject.Subject.proj('sex') * query & s).fetch('sex', 'weight','session_performance')
    
    weight = weight[performance > 0]
    performance = performance[performance > 0]
    if sex[0] == 'Male':
        pl.scatter(weight, performance, c=[[0, 0, 1]])
    else: 
        pl.scatter(weight, performance, c=[[1, 0, 0]])
    corrs.append(np.corrcoef(weight, performance)[0, 1])
    
pl.xlim([19, 27])
pl.xlabel('bodyweight [g]')
pl.ylabel('performance')

In [ ]:
subjects = (subject.Subject() & 'sex = "Male"' & 'user_id = "edward"' ).fetch('KEY')
boymean = np.mean((Session & subjects).fetch('session_performance'))
subjects = (subject.Subject() & 'sex = "Female"' & 'user_id = "edward"' ).fetch('KEY')
girlmean = np.mean((Session & subjects).fetch('session_performance'))
print(boymean)
print(girlmean)

## Correlation between weight and performance for all mice

In [ ]:
pl.hist(corrs,np.arange(-1,1,0.1))
print(np.mean(corrs))
print(np.std(corrs)/np.sqrt(len(corrs)))

# Compute trial numbers for all sessions of a mouse

In [ ]:
sessions = acquisition.Session & 'subject_id = "E39"' & 'session_date > "2017-10-21"'
sessions

In [ ]:
trials_subset = acquisition.TowersBlock.Trial & sessions
acquisition.Session.aggr(trials_subset, n='count(*)')

# Visualize all trials for a session

In [ ]:
pos, time = trials_subset.fetch('position', 'trial_time')

In [ ]:
for trial_idx in range(len(pos)):
    N_samples = len(pos[trial_idx])
    pl.plot(time[trial_idx][0:N_samples], pos[trial_idx][:,1])
pl.xlabel('Time [s]');
pl.ylabel('position [cm]');

In [ ]:
query = acquisition.TowersBlock & 'subject_id="E39"' & 'session_date = "2017-08-04"'

In [ ]:
acquisition.TowersBlock.Trial & query